In [1]:
import poppy
import jax
import jax.numpy as jnp
import functools

In [3]:
p = 11
n = 3
f = poppy.field( p, n )
f.X, f.INV

(Array([[[1, 0, 0],
         [0, 1, 0],
         [0, 0, 1]],
 
        [[0, 0, 2],
         [1, 0, 9],
         [0, 1, 0]],
 
        [[0, 2, 0],
         [0, 9, 2],
         [1, 0, 9]]], dtype=int64),
 Array([ 0,  1,  6,  4,  3,  9,  2,  8,  7,  5, 10], dtype=int64))

In [3]:
d = 4
a = poppy.random( ( d, d ), f, poppy.SEED + 0 )
b = poppy.random( ( d, d ), f, poppy.SEED + 1 )

In [4]:
a.shape, a.lift.shape, a.lift

((4, 4),
 (12, 12),
 Array([[2, 4, 1, 3, 0, 3, 3, 0, 0, 1, 3, 0],
        [3, 1, 0, 4, 3, 3, 0, 3, 0, 0, 4, 3],
        [2, 3, 1, 0, 4, 3, 0, 0, 3, 4, 0, 4],
        [0, 0, 4, 0, 0, 3, 3, 3, 1, 3, 0, 3],
        [2, 0, 4, 4, 0, 3, 3, 1, 4, 4, 3, 3],
        [0, 2, 0, 0, 4, 0, 4, 3, 1, 0, 4, 3],
        [4, 4, 3, 0, 1, 3, 4, 2, 4, 1, 3, 4],
        [4, 3, 2, 4, 1, 4, 2, 1, 1, 2, 4, 2],
        [2, 4, 3, 3, 4, 1, 1, 2, 1, 4, 2, 4],
        [0, 1, 0, 4, 2, 0, 1, 0, 0, 1, 4, 0],
        [0, 1, 1, 0, 1, 2, 0, 1, 0, 0, 0, 4],
        [3, 0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 0]], dtype=int64))

In [5]:
ai = a.inv( )
bi = b.inv( )

In [6]:
print( a.lift[ 0, : 10 ] )
( a - a @ ai @ a @ bi @ b ).lift

[2 4 1 3 0 3 3 0 0 1]


Array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [7]:
@functools.partial( jax.jit, static_argnums = ( 1, 2 ) )
def poppyhash( a, F, FF ):
    b = poppy.proj( a, F )
    c = poppy.proj( b, FF )
    return c

In [8]:
liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash,  in_axes = ( 0, None, None ) )

In [9]:
p  = 5
n  = 6
d  = 10
N  = 1000
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED ), ( N, d, d ), 0, p ** n, dtype = jnp.int64 )
ap = liftbatch( a, poppy.field( p, n ) )
F  = poppy.field( 31, n )
FF = poppy.field( 7, d )
h  = hashbatch( ap, F, FF ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu )

(1000, 1000)